In [1]:
%pylab inline
import pandas as pd
import numpy as np
from selenium import webdriver
import time
from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    # path = '~/Documents/GitHub/f1-analytics/'
    path = '/Users/oliverjcarter/Documents/GitHub/f1-analytics/'

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
races = pd.read_csv(path+'data/races.csv')

In [3]:
races.head()

,season,round,circuit_id,country,lat,long,url,date
0,2014,1,albert_park,Australia,-37.84970,144.96800,http://en.wikipedia.org/wiki/2014_Australian_G...,2014-03-16 06:00:00
1,2014,2,sepang,Malaysia,2.76083,101.73800,http://en.wikipedia.org/wiki/2014_Malaysian_Gr...,2014-03-30 08:00:00
2,2014,3,bahrain,Bahrain,26.03250,50.51060,http://en.wikipedia.org/wiki/2014_Bahrain_Gran...,2014-04-06 15:00:00
3,2014,4,shanghai,China,31.33890,121.22000,http://en.wikipedia.org/wiki/2014_Chinese_Gran...,2014-04-20 07:00:00
4,2014,5,catalunya,Spain,41.57000,2.26111,http://en.wikipedia.org/wiki/2014_Spanish_Gran...,2014-05-11 12:00:00


In [ ]:
weather = races.iloc[:,[0,1,2]]

In [ ]:
info = []

for link in races.url:
    try:
        df = pd.read_html(link)[0]
        if 'Weather' in list(df.iloc[:,0]):
            n = list(df.iloc[:,0]).index('Weather')
            info.append(df.iloc[n,1])
        else:
            df = pd.read_html(link)[1]
            if 'Weather' in list(df.iloc[:,0]):
                n = list(df.iloc[:,0]).index('Weather')
                info.append(df.iloc[n,1])
            else:
                df = pd.read_html(link)[2]
                if 'Weather' in list(df.iloc[:,0]):
                    n = list(df.iloc[:,0]).index('Weather')
                    info.append(df.iloc[n,1])
                else:
                    df = pd.read_html(link)[3]
                    if 'Weather' in list(df.iloc[:,0]):
                        n = list(df.iloc[:,0]).index('Weather')
                        info.append(df.iloc[n,1])
                    else:
                        driver = webdriver.Chrome()
                        driver.get(link)

                        # click language button
                        button = driver.find_element_by_link_text('Italiano')
                        button.click()
                        
                        clima = driver.find_element_by_xpath('//*[@id="mw-content-text"]/div/table[1]/tbody/tr[9]/td').text
                        info.append(clima) 
                                
    except:
        info.append('not found')

In [ ]:
weather['weather'] = info

In [ ]:
weather.head()

In [ ]:
weather_dict = {'weather_warm': ['soleggiato', 'clear', 'warm', 'hot', 'sunny', 'fine', 'mild', 'sereno'],
               'weather_cold': ['cold', 'fresh', 'chilly', 'cool'],
               'weather_dry': ['dry', 'asciutto'],
               'weather_wet': ['showers', 'wet', 'rain', 'pioggia', 'damp', 'thunderstorms', 'rainy'],
               'weather_cloudy': ['overcast', 'nuvoloso', 'clouds', 'cloudy', 'grey', 'coperto']}

In [ ]:
weather_df = pd.DataFrame(columns = weather_dict.keys())

In [ ]:
for col in weather_df:
    weather_df[col] = weather['weather'].map(lambda x: 1 if any(i in weather_dict[col] for i in x.lower().split()) else 0)

In [ ]:
weather_df.head()